In [1]:
import time
import sys
sys.path.append('../')
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait()

In [2]:
%%time
engine = pyttsx3.init()
engine.say("Processing featuring of dataframes, weekly, daily and intra-day")
engine.runAndWait()
TICKER_LIST = get_ticker_list()
df_all_15 = get_all_data(TICKER_LIST,_period='m15')
hourly_all = get_all_data(TICKER_LIST,_period='H1')
hourly_all = timerange1D(hourly_all)
df_all_15 = timerange1D(df_all_15)
daily_all = get_daily(hourly_all,TICKER_LIST)
daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("The job is done")
engine.runAndWait()

100%|██████████| 42/42 [00:00<00:00, 176513.80it/s]
Récupération des tickers

Récupération des data intraday

100%|██████████| 42/42 [00:00<00:00, 156032.57it/s]
Récupération des data intraday


Ajout TimeRange

Ajout TimeRange
100%|██████████| 42/42 [00:00<00:00, 160291.87it/s]
Récupération des tickers daily


Ajout colonne 'Date dans le weekly
100%|██████████| 42/42 [00:00<00:00, 240001.05it/s]
Récupération des tickers weekly

CPU times: user 1min 46s, sys: 7.7 s, total: 1min 53s
Wall time: 4min 37s


In [3]:
%%time
daily_all = adr(daily_all,_window=14)

 14%|█▍        | 6/42 [00:00<00:00, 54.82it/s]
Calcul du ADR
100%|██████████| 42/42 [00:00<00:00, 43.74it/s]
CPU times: user 1.02 s, sys: 35.3 ms, total: 1.06 s
Wall time: 1.05 s


In [4]:
%%time
#### CHOIX DE LA TF QUI SERA TESTEE
df_all_15 = getadr(daily_all,df_all_15,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 257544.98it/s]
CPU times: user 2min 36s, sys: 24.6 s, total: 3min 1s
Wall time: 5min 43s


In [5]:
%%time
#### CHOIX DE LA TF QUI SERA TESTEE
df_all_15 = adrhnl(daily_all,df_all_15,TICKER_LIST)


Calcul du ADR High & Low
100%|██████████| 42/42 [00:00<00:00, 130103.96it/s]
CPU times: user 2min 18s, sys: 39.4 s, total: 2min 58s
Wall time: 7min 9s


In [39]:
%%time
df_all_15 = rsi(df_all_15,_fast=2,_slow=14)


Calcul RSI
CPU times: user 21.9 s, sys: 6.43 s, total: 28.4 s
Wall time: 27.8 s


In [53]:
%%time
df_all_15 = jyss_oscillator(df_all_15])

100%|██████████| 1/1 [00:00<00:00, 10754.63it/s]
Calcul du JYSS OSCILLATOR

Création du Jyss Oscillator
HiWin => plus haut High d'une fenêtre de 10
LoWin => plus bas Low d'une fenêtre de 10
JyssOscBear => (HiWin - HiWin.shift(2))/HiWin.shift(2
JyssOscBull => (LoWin - LoWin.shift(2))/LoWin.shift(2
JyssOscBearSD et JyssOscBullSD => Les std() sur des fenêtres de 20
TriggerBear => (HiWin - HiWin.shift(2) * 1000
TriggerBull => (LoWin - LoWin.shift(2) * 1000
Trigger => 1 quand il y a égalité entre LoWin et LoWin.shift(9) ou quand il y a égalité entre HiWin et HiWin.shift(9
SigJyssOsc => 1 quand (Close <= LoWin) & (JyssOscUP<= -4 * JyssOscUpSD ) ou -1 quand (Close >= HiWin) & (JyssOscDwn >= 4 * JyssOscDwnSD )


CPU times: user 917 ms, sys: 393 ms, total: 1.31 s
Wall time: 2.7 s


In [47]:
%%time
#### CONDITIONS LONG
_condition_1 = df_all_15.Low < df_all_15.ADR_Low # adrlo
_condition_2 = df_all_15.Close >=df_all_15.ADR_Low # close adrlo
_condition_3 = df_all_15.RSI_14 <= 30 # RSI

_condition_1_hat = df_all_15.High > df_all_15.ADR_High # adrhi
_condition_2_hat = df_all_15.Close < df_all_15.ADR_High # close adrhi
_condition_3_hat = df_all_15.RSI_14 >= 70


##### 1 condition
#df_all_15['Signal'] = np.where(_condition_1,1,np.where(_condition_1_hat,-1,0))

##### 2 conditions
df_all_15['Signal'] = np.where(((_condition_1)&(_condition_3)),1,np.where(((_condition_1_hat)&(_condition_3_hat)),-1,0))


##### 2 conditions
#df_all_15['Signal'] = np.where(((_condition_1)&(_condition_2)&(_condition_3)),1,np.where(((_condition_1_hat)&(_condition_2_hat)&(_condition_3_hat)),-1,0))


##### 2 conditions
#df_all_15['Signal'] = np.where(((_condition_1)&(_condition_2)&(_condition_3)&(_condition_4)),1,np.where(((_condition_1_hat)&(_condition_2_hat)&(_condition_3_hat)&(_condition_4_hat)),-1,0))

CPU times: user 455 ms, sys: 21.1 ms, total: 476 ms
Wall time: 142 ms


In [7]:
from librairies.bt import *

Importing Librairies...


In [90]:
x='EUR/USD'
_year_bottom = '2019-01-01'
_year_top = '2019-12-31'
_nb_bougie_exit = 1200000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 0
_verbose = 0
_cash_ini = 200000
_rate = 1
hourly = df_all_15[df_all_15.Symbol == x.replace('/','')]
_target = 0.002
_exposure = 10
_size = 50000
_sl = 0.003
_open_hour = 9
_close_hour = 22
_window = 0

TRACKER = bt_dayonly(hourly,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl,_open_hour,_close_hour,_window)

#TRACKER = bt(hourly,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
 #       _rate,x,_target,_exposure,_size,_sl)

Librairies imported

Début des opérations horodatée à 2021-01-19 13:34:45.243413

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 23823/23823 [00:12<00:00, 1854.93it/s]
 Pour le ticker  EUR/USD 
 
Les gains faramineux s'élèvent à : $  -1313.0 !. En  106   transactions.
 
Nbre de winners : 51 
 
Nbre de loosers : 55 
Temps d'excution du BT 0:00:19.136669
 EUR/USD  results 
 Tested Period 2019-01-01  à 2019-12-31 
 Total Number of trades 106 
Started Cash : 50000
P&L in currency:  -1313.0$ 
P&L in %:  -0.66% 
Average trade duration 37.65
# Winners  51.0
# Loosers  55.0
Cumulated gains 3764.0
Cumulated losses -5077.0
 PROFIT FACTOR :  0.74 
 Winners Ratio : 48.11 % 
Average Winners 73.8
% Average Winners 0.04
Average Loosers -92.31
% Average Loosers -0.05
Average pnl -12.39
% Average pnl -0.01
Number of opened trades 106
Number of closed trades 106
Max Exposure 1 x  50000 =  50000 $
